In [110]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import string
import re
import nltk
import sklearn
%matplotlib inline

In [82]:
clean_data = pd.read_excel('clean.xlsx')

In [83]:
clean_data = clean_data.iloc[np.random.permutation(len(clean_data))]

In [84]:
clean_data = clean_data.reset_index(drop=True)

# Making bag of words model

In [85]:
from sklearn.feature_extraction.text import CountVectorizer

allthetext = []
for i in xrange(len(clean_data)):
    allthetext.append(clean_data['Text'][i])

In [86]:
len(allthetext)

21768

In [87]:
allthewords = {}

for text in allthetext:
    for word in text.split():
        if word in allthewords:
            allthewords[word] = allthewords[word] + 1
        else:
            allthewords[word] = 1
    


In [88]:
import operator
sorted_all = sorted(allthewords.items(), key=operator.itemgetter(1))

In [89]:
sorted_all.reverse()

In [90]:
len(allthewords)

58682

# There are 58683 unique words, and I have to select which are the most important ones(by frequency) for feature

In [91]:
np.unique(clean_data['Category'])

array([u'Canola', u'Corn', u'Hay & Forage', u'Pulse', u'Soybeans', u'Wheat'], dtype=object)

In [92]:
subset_sorted = [(a,b) for (a,b) in sorted_all if b>1000]

In [93]:
len(subset_sorted)

1065

# Seems like 1000 frequency threshold gives 1066 unique words, lets keep that for now

In [94]:
feature_words = [a for (a,b) in subset_sorted]

In [95]:
vectorizer = CountVectorizer(analyzer='word', tokenizer=None, preprocessor=None, stop_words=None, max_features=1065)

In [96]:
clean_text_list = []
for i in xrange(len(clean_data)):
    clean_text_list.append(clean_data['Text'][i])

In [97]:
train_features = vectorizer.fit_transform(clean_text_list)

In [98]:
vocab = vectorizer.get_feature_names()
#print(vocab)

In [99]:
train_features = train_features.toarray()

In [100]:
train_features.shape

(21768, 1065)

# Creating a training, validation and test split

In [101]:
from sklearn.cross_validation import train_test_split

data_train_and_validation, data_test, labels_train_and_validation, labels_test = train_test_split(train_features, clean_data['Category'], test_size = 0.10, random_state = 0)

In [102]:
data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_and_validation, labels_train_and_validation, test_size = 0.20, random_state = 0)

In [103]:
print(len(data_train))
print(len(data_validation))
print(len(data_test))

15672
3919
2177


In [104]:
print(len(labels_train))
print(len(labels_validation))
print(len(labels_test))

15672
3919
2177


# Trying out out of the box Random Forest model

In [112]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 200)

forest = forest.fit(data_train, labels_train)

In [113]:
results_test = forest.predict(data_test)

In [114]:
sklearn.metrics.accuracy_score(labels_test, results_test, normalize = True)

0.54065227377124481